In [22]:
import cv2
from ultralytics import YOLO
import pandas as pd
import json
import os

video_path = "traffic4.mp4"
output_video_path = "output.mp4"
output_csv_path = "results.csv"
output_json_path = "results.json"

model = YOLO("yolov8n.pt")
cap = cv2.VideoCapture(video_path)

target_size = (320, 180)

fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter(output_video_path,
                      cv2.VideoWriter_fourcc(*'mp4v'),
                      fps,
                      target_size)

frame_count = 0
log_data = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, target_size)
    frame_count += 1
    timestamp = round(frame_count / fps, 2)

    results = model.predict(source=frame, device='cpu')[0]

    car_count = 0
    bus_count = 0
    truck_count = 0
    person_count = 0

    for box in results.boxes.data.tolist():
        x1, y1, x2, y2, conf, cls = box
        cls = int(cls)
        label = None
        color = (255, 255, 255)

        if cls == 0: 
            person_count += 1
            label = f"Person {conf:.2f}"
            color = (255, 255, 0)
        elif cls == 2: 
            car_count += 1
            label = f"Car {conf:.2f}"
            color = (0, 255, 0)
        elif cls == 5:
            bus_count += 1
            label = f"Bus {conf:.2f}"
            color = (0, 165, 255)
        elif cls == 7: 
            truck_count += 1
            label = f"Truck {conf:.2f}"
            color = (255, 0, 0)

        if label:
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 1)
            cv2.putText(frame, label, (int(x1), int(y1) - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)

    out.write(frame)

    log_data.append({
        "timestamp_sec": timestamp,
        "car_count": car_count,
        "bus_count": bus_count,
        "truck_count": truck_count,
        "person_count": person_count
    })

cap.release()
out.release()

df = pd.DataFrame(log_data)
df.to_csv(output_csv_path, index=False)

with open(output_json_path, 'w') as f:
    json.dump(log_data, f, indent=4)

print(f"Done! Video saved to {output_video_path}")
print(f"Results saved to {output_csv_path} and {output_json_path}")



0: 384x640 4 persons, 3 cars, 66.4ms
Speed: 4.8ms preprocess, 66.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 3 cars, 73.9ms
Speed: 2.0ms preprocess, 73.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 3 cars, 70.1ms
Speed: 5.8ms preprocess, 70.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 3 cars, 1 truck, 64.7ms
Speed: 2.1ms preprocess, 64.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 4 cars, 58.8ms
Speed: 2.4ms preprocess, 58.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 2 cars, 56.0ms
Speed: 1.8ms preprocess, 56.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 3 cars, 61.4ms
Speed: 1.5ms preprocess, 61.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 4 cars, 68.3ms
Speed: 2


0: 384x640 2 persons, 3 cars, 1 bus, 1 truck, 79.5ms
Speed: 3.2ms preprocess, 79.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 4 cars, 1 truck, 67.9ms
Speed: 1.5ms preprocess, 67.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 1 truck, 72.5ms
Speed: 3.1ms preprocess, 72.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 1 truck, 63.6ms
Speed: 1.6ms preprocess, 63.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 cars, 1 truck, 58.8ms
Speed: 1.6ms preprocess, 58.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 2 cars, 1 bus, 1 truck, 58.4ms
Speed: 2.9ms preprocess, 58.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 cars, 1 truck, 54.1ms
Speed: 2.2ms preprocess, 54.1ms inference, 1.0ms postprocess per image at shap


0: 384x640 2 persons, 4 cars, 1 bus, 1 truck, 62.2ms
Speed: 1.5ms preprocess, 62.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 4 cars, 1 bus, 1 truck, 1 handbag, 57.8ms
Speed: 2.2ms preprocess, 57.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 5 cars, 1 train, 1 truck, 1 handbag, 64.2ms
Speed: 1.7ms preprocess, 64.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 4 cars, 1 truck, 61.5ms
Speed: 3.2ms preprocess, 61.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 4 cars, 1 bus, 1 truck, 59.9ms
Speed: 3.5ms preprocess, 59.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 4 cars, 1 truck, 60.2ms
Speed: 2.0ms preprocess, 60.2ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 4 cars, 1 bus, 1 truck, 81.9ms
Speed: 5.5ms preprocess, 81.9ms i

Speed: 2.6ms preprocess, 89.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 4 cars, 1 bus, 72.9ms
Speed: 3.3ms preprocess, 72.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 cars, 1 bus, 68.3ms
Speed: 2.8ms preprocess, 68.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 1 bus, 82.2ms
Speed: 2.1ms preprocess, 82.2ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 4 cars, 1 bus, 97.7ms
Speed: 3.7ms preprocess, 97.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 5 cars, 1 bus, 71.2ms
Speed: 4.7ms preprocess, 71.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 cars, 1 bus, 78.5ms
Speed: 2.3ms preprocess, 78.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 5 cars, 1 bus, 1 truck, 127.2ms



0: 384x640 1 person, 4 cars, 1 bus, 94.3ms
Speed: 3.1ms preprocess, 94.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 4 cars, 1 bus, 80.0ms
Speed: 3.6ms preprocess, 80.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 4 cars, 1 bus, 71.7ms
Speed: 1.9ms preprocess, 71.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 4 cars, 1 bus, 1 truck, 72.7ms
Speed: 2.1ms preprocess, 72.7ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 1 bus, 1 truck, 86.5ms
Speed: 3.2ms preprocess, 86.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 1 bus, 1 truck, 1 stop sign, 105.6ms
Speed: 3.6ms preprocess, 105.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 cars, 1 bus, 2 trucks, 87.6ms
Speed: 3.4ms preprocess, 87.6ms inference, 1.0ms postproce

...


0: 384x640 1 person, 3 cars, 1 truck, 1 handbag, 70.8ms
Speed: 3.3ms preprocess, 70.8ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 5 cars, 1 truck, 1 handbag, 64.3ms
Speed: 1.7ms preprocess, 64.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 2 cars, 62.7ms
Speed: 2.3ms preprocess, 62.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 cars, 80.7ms
Speed: 3.1ms preprocess, 80.7ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 cars, 78.0ms
Speed: 2.1ms preprocess, 78.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 78.9ms
Speed: 2.3ms preprocess, 78.9ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 69.4ms
Speed: 1.9ms preprocess, 69.4ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 83.8ms
Sp


0: 384x640 2 cars, 1 bus, 2 trucks, 109.2ms
Speed: 2.4ms preprocess, 109.2ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 bus, 2 trucks, 101.3ms
Speed: 2.5ms preprocess, 101.3ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 2 trucks, 103.6ms
Speed: 3.2ms preprocess, 103.6ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 2 trucks, 107.9ms
Speed: 2.9ms preprocess, 107.9ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 2 trucks, 119.1ms
Speed: 3.3ms preprocess, 119.1ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 112.7ms
Speed: 2.6ms preprocess, 112.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 bus, 2 trucks, 109.7ms
Speed: 4.2ms preprocess, 109.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 truck, 15

Speed: 2.5ms preprocess, 65.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 car, 1 truck, 71.4ms
Speed: 2.2ms preprocess, 71.4ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 car, 1 truck, 78.0ms
Speed: 2.2ms preprocess, 78.0ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 car, 2 trucks, 75.6ms
Speed: 2.3ms preprocess, 75.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 car, 2 trucks, 104.9ms
Speed: 3.1ms preprocess, 104.9ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 car, 1 truck, 100.7ms
Speed: 2.4ms preprocess, 100.7ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 car, 1 truck, 92.4ms
Speed: 2.2ms preprocess, 92.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 car, 1 truck, 97.3ms



0: 384x640 1 car, 1 bus, 1 train, 1 truck, 110.7ms
Speed: 2.0ms preprocess, 110.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 bus, 1 train, 1 truck, 90.0ms
Speed: 2.5ms preprocess, 90.0ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 bus, 1 truck, 91.3ms
Speed: 2.0ms preprocess, 91.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 truck, 118.4ms
Speed: 1.8ms preprocess, 118.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 bus, 1 truck, 87.9ms
Speed: 2.9ms preprocess, 87.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 bus, 1 truck, 71.0ms
Speed: 3.8ms preprocess, 71.0ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 79.5ms
Speed: 2.3ms preprocess, 79.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 ca


0: 384x640 2 persons, 1 car, 1 bus, 1 train, 83.4ms
Speed: 1.7ms preprocess, 83.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 1 car, 1 bus, 1 train, 91.2ms
Speed: 3.2ms preprocess, 91.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 car, 1 train, 121.0ms
Speed: 2.0ms preprocess, 121.0ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 car, 1 bus, 1 train, 111.9ms
Speed: 4.1ms preprocess, 111.9ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 car, 1 bus, 1 train, 104.8ms
Speed: 2.9ms preprocess, 104.8ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 car, 1 train, 111.3ms
Speed: 2.2ms preprocess, 111.3ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 car, 1 bus, 101.2ms
Speed: 2.7ms preprocess, 101.2ms inference, 1.6ms postprocess p